## League of Legends - Data Processing Notebook

Purpose: This notebook focuses strictly on acquiring and persisting the RAW Riot match data (match detail + timeline). No cleaning, feature engineering, or train/val/test splitting happens here — those steps live in the DAIA and Modeling notebook.

Scope in this notebook:
- Authenticate with Riot API and fetch raw match JSONs to `data/raw/`.
- Maintain a lightweight `matchlist.json` to track which matches are available locally.

Out of scope (handled in DAIA & Modeling):
- Building pregame or 15-minute feature tables.
- Cleaning, filtering, or schema validation.
- Train/validation/test splits and modeling.

## Introduction & Goals

**Goals of this notebook:**

-   Collect match data from the Riot API.

-   Parse and extract both pre-game features (champion picks, roles) and early-game features at 15 minutes (gold diff, kills, objectives).

-   Produce a cleaned, versioned dataset (CSV) with clear documentation for each column.

**Why separate processing from modeling?**

Riot match timelines and match lists can become large (MBs–GBs). Processing and cleaning in a dedicated notebook or script avoids repeated heavy work during model experimentation and keeps the ML notebook focused on training and evaluation.

# Explainable AI (XAI)

### Key Concepts
- XAI = Explainable Artificial Intelligence; methods and techniques to make AI models transparent and understandable to humans.
- **Global explanations**: Show which features are most influential on average across a dataset.
- **Local explanations**: Explain why a specific prediction was made.
- **Intrinsic vs Post-hoc**: 
  - **Intrinsic**: Models that are transparent by design (e.g., decision trees, linear models).  
  - **Post-hoc**: Methods applied to black-box models after training (e.g., SHAP, LIME, counterfactuals).
- **Visualization & communication**: Plots, summary graphs, or textual explanations make insights accessible for different audiences.

### Pros of XAI
- **Transparency:** Users understand why a model makes a prediction.
- **Trust:** Increases adoption by showing clear reasoning.
- **Debugging & improvement:** Identifies if models rely on meaningful or misleading signals.
- **Actionable insights:** Helps make decisions in strategy, operations, or coaching.
- **Flexible communication:** Tailored explanations for technical and non-technical users.

### Cons and Dangers
- **Complexity:** Adds extra steps to modeling pipelines.
- **Misinterpretation:** Oversimplified explanations may mislead users.
- **Trade-off with accuracy:** Simple interpretable models may be less accurate; black-box models need complex post-hoc explanations.
- **Computational cost:** Methods like SHAP can be resource-intensive on large datasets.
- **Partial explanations:** Some methods only approximate reasoning, which can be risky if users over-trust them.
- **Ethical & safety concerns:** Revealing certain features or internal logic may expose vulnerabilities (e.g., security risks, sensitive data leaks).

## Application Across Domains

#### Games (General)
- XAI is used to explain AI behavior in a wide range of games, from strategy to simulation:  
  - **NPC behavior**: Why an AI-controlled character made a certain move or decision.  
  - **Dynamic difficulty adjustment (DDA)**: Why the game adapts difficulty based on player performance.  
  - **Analytics and predictions**: Explaining why an AI predicts a certain outcome or strategy is likely to succeed.  
- Benefits: Helps developers debug and balance games, improves player trust, and provides educational insight into gameplay mechanics.  
- Challenges: High-dimensional state spaces, real-time decision-making, and latent human factors like coordination or communication are hard to capture.

#### League of Legends (Specific)
- Early-game gold, first tower, and objectives are strong predictors of match outcomes.  
- XAI helps **coaches, players, and analysts** understand *why* a team is favored.  
- Limitations: Some factors like team synergy, communication, or morale cannot be measured numerically.  
- Goal: Make predictions interpretable and actionable in real time without compromising fairness or strategy secrecy.

#### Science and Engineering
- XAI explains results from simulations, experiments, or predictive maintenance models.
- Enables **trust, safety, and accountability**: Users must understand why models make predictions that impact real-world systems.
- Examples: AI flagging faulty machinery, medical diagnosis, or chemical simulations.
- Challenge: Balancing **high performance** (accuracy) and **interpretability** is critical for safety.

#### Ethical and Regulatory Considerations
- XAI is increasingly required for ethical, legal, and safety compliance (e.g., GDPR in Europe, AI safety standards).
- Certain aspects of models should **not be fully disclosed** to prevent misuse:
  - Trade secrets or proprietary algorithms.
  - Sensitive personal data in healthcare or finance.
  - Safety-critical logic where exposing details could be dangerous (e.g., autonomous vehicles or critical infrastructure).
- Regulation ensures transparency without compromising security or privacy.

### What to Explain and What Not to Explain
- **Explain:**  
  - Key predictive features and their contributions.  
  - Model reasoning that impacts human decision-making.  
  - Patterns that help users learn, plan, or optimize strategy.
- **Do not explain fully:**  
  - Sensitive data or personally identifiable information.  
  - Internal logic that could create security risks.  
  - Proprietary algorithms where disclosure is not allowed.

### Personal Learning Takeaways
- XAI bridges the gap between AI predictions and human understanding, making models more trustworthy and actionable.
- Choosing the right model depends on **accuracy, interpretability, and domain needs**.
- Safety, ethics, and regulations are as important as performance: what we explain and what we hide must be carefully considered.
- Applications differ by domain: in games, clarity and strategic insight are priorities; in science or engineering, safety, reliability, and reproducibility are critical.


## Environment & setup

In [4]:
# Import core Python libraries
import pandas as pd
import requests
import tqdm
import pyarrow
import fastparquet
import matplotlib.pyplot as plt
import seaborn as sns
import os
import time
import json
from typing import Dict, Any, List
from dotenv import load_dotenv

import requests
import pandas as pd
from tqdm import tqdm


# plotting
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(context="notebook", style="darkgrid", palette="deep", font="sans-serif", font_scale=1)


# Load environment variables
from dotenv import load_dotenv
load_dotenv()


RIOT_API_KEY = os.getenv('RIOT_API_KEY')
if not RIOT_API_KEY:
    raise RuntimeError('Set RIOT_API_KEY as an environment variable or in a .env file')


HEADERS = {"X-Riot-Token": RIOT_API_KEY}

## Data sourcing (Riot API overview)

### What endpoints we be used:

- **Match IDs** by **PUUID:** /lol/match/v5/matches/by-puuid/{puuid}/ids - returns a list of match IDs for a given player.

- **Match detail by matchId**: /lol/match/v5/matches/{matchId} — returns the full match info and timeline (timelines might be included alongside).

### Filtering notes:

- Use **queue=420** to restrict to **Ranked Solo/Duo matches (competitive).**

- Filter out **remakes** or **extremely short games** (e.g., gameDuration < 9 minutes) to avoid noisy examples.

### Rate limits & best practices:

- Riot enforces **rate limits**, so I will be using **time.sleep** to pause between requests and wait longer if blocked (HTTP 429).

- **Cache** raw match JSONs to disk so you don't re-download them.

## Data requirements and Schema


### 1. Core Features (and why)

- **Match metadata**: `matchId`, `queueId`, `gameDuration` - needed to filter matches.
- **Champion & role picks**: champion IDs/names, team positions - needed for pre-game features.
- **Lane metrics at 15 minutes**: gold diff, CS diff, XP diff, K/D/A per lane - strong early indicators.
- **Team objectives by 15 minutes**: `firstTower`, `firstDragon`, `firstHerald`, void grubs (`horde`).
- **Target variables**: `blue_win` (0/1) and `win_probability` (for model output).



### 2. Proposed Output Schema (one row per match)

### Match Metadata
- `matchId` (`str`)
- `queueId` (`int`)
- `gameDuration` (`int`, seconds)
- `blue_win` (`0/1`)

### Champion Picks & Roles
- `blue_champions` (list of champion IDs)
- `red_champions` (list of champion IDs)
- `blue_roles` (list of positions)
- `red_roles` (list of positions)

### Early Features (15 min)
- `blue_gold_15`, `red_gold_15`, `gold_diff_15` - difference in gold at 15 minutes
- `blue_cs_15`, `red_cs_15`, `cs_diff_15` - difference in farm at 15 minutes
- `blue_xp_15`, `red_xp_15`, `xp_diff_15` - difference in XP at 15 minutes
- `blue_kills_15`, `red_kills_15`, `kills_diff_15` - difference in kills at 15 minutes

### Objectives (15 min)
- `first_tower` (None / blue / red)
- `first_dragon` (None / blue / red)
- `first_herald` (None / blue / red)
- `first_grub` (None / blue / red)



## Storage strategy & versioning

For this project, storage is intentionally simple in this notebook: we cache RAW JSON responses under `data/raw/` and maintain `matchlist.json`. Processed/cleaned tables are created downstream in the DAIA & Modeling notebook.

- Raw data: every match JSON from the Riot API lives in `data/raw/` (both match detail and timeline). We never overwrite these so we can always reprocess.
- Match index: `matchlist.json` lists the match IDs we have locally.

Note: Processed artifacts (Parquet/CSV) will be written by the DAIA notebook, not here. This keeps acquisition (this notebook) separate from processing/modeling (DAIA).

## Raw data collection

### Important initial design choices:

- I am going to collect match IDs by querying several PUUIDs (players). Prefer high-activity public accounts.

- For initial proof-of-concept, I will collect 1,000–5,000 matches. For final model I will have tens of thousands

In [10]:
# Constants + basic Riot fetch helpers (simple version)
REGION = 'europe'  # riot regional routing
HEADERS = {"X-Riot-Token": RIOT_API_KEY}  # auth header
MATCHLIST_URL = f'https://{REGION}.api.riotgames.com/lol/match/v5/matches/by-puuid/{{puuid}}/ids'
MATCH_URL = f'https://{REGION}.api.riotgames.com/lol/match/v5/matches/{{matchId}}'
TIMELINE_URL = f'https://{REGION}.api.riotgames.com/lol/match/v5/matches/{{matchId}}/timeline'
RAW_DIR = 'data/raw'

# Basic GET with tiny retry + rate limit handling
def safe_get(url, params=None, retries=5):
    for i in range(retries):
        resp = requests.get(url, headers=HEADERS, params=params)
        if resp.status_code == 200:
            return resp.json()
        elif resp.status_code == 429:  # rate limited
            wait = int(resp.headers.get('Retry-After', 1))
            print(f"Rate limited. Waiting {wait}s...")
            time.sleep(wait)
    raise RuntimeError(f"Failed to GET {url} after {retries} retries")

# Get player unique id from riot name + tag
def get_puuid(game_name, tag_line):
    url = f"https://{REGION}.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{game_name}/{tag_line}"
    resp = requests.get(url, headers=HEADERS)
    if resp.status_code == 200:
        return resp.json().get("puuid")
    print(f"Failed to get PUUID for {game_name}#{tag_line}: {resp.status_code}")
    return None

# Just grab some match ids for one player (solo queue)
def fetch_match_ids(puuid, count=20, queue=420):
    params = {"start": 0, "count": count, "queue": queue}
    return safe_get(MATCHLIST_URL.format(puuid=puuid), params=params)

# Save both match + timeline JSON locally if missing (cache)
def fetch_and_save_match_with_timeline(match_id):
    os.makedirs(RAW_DIR, exist_ok=True)

    match_path = os.path.join(RAW_DIR, f"{match_id}.json")
    if not os.path.exists(match_path):  # don't redownload
        match_data = safe_get(MATCH_URL.format(matchId=match_id))
        with open(match_path, 'w', encoding='utf-8') as f:
            json.dump(match_data, f)

    timeline_path = os.path.join(RAW_DIR, f"{match_id}_timeline.json")
    if not os.path.exists(timeline_path):
        timeline_data = safe_get(TIMELINE_URL.format(matchId=match_id))
        with open(timeline_path, 'w', encoding='utf-8') as f:
            json.dump(timeline_data, f)

    return match_path, timeline_path

# Generic fetch/save helper
def fetch_and_save_json(url, path):
    os.makedirs(os.path.dirname(path), exist_ok=True)
    if os.path.exists(path):  # already there
        return path
    data = safe_get(url)
    with open(path, 'w', encoding='utf-8') as f:
        json.dump(data, f)
    return path

# Single test player (can add more later)
players = [{"name": "BB99", "tag": "BLZNT"}]

# Resolve their PUUIDs
for p in players:
    p["puuid"] = get_puuid(p["name"], p["tag"])
puuids = [p["puuid"] for p in players if p.get("puuid")]
print("PUUIDs fetched:", puuids)

# Rate limiting guard (rough)
all_match_ids = []
MAX_REQUESTS = 99  # under 100 per 2 min window
WINDOW = 120
request_count = 0
start_time = time.time()

for puuid in puuids:
    match_ids = fetch_match_ids(puuid, count=20)  # small sample
    all_match_ids.extend(match_ids)
    
    for match_id in tqdm(match_ids, desc=f"Downloading matches for {puuid}"):
        # Full match
        fetch_and_save_json(MATCH_URL.format(matchId=match_id), os.path.join(RAW_DIR, f"{match_id}.json"))
        # Timeline
        fetch_and_save_json(TIMELINE_URL.format(matchId=match_id), os.path.join(RAW_DIR, f"{match_id}_timeline.json"))
        
        request_count += 2  # two calls per match
        if request_count >= MAX_REQUESTS:
            elapsed = time.time() - start_time
            sleep_time = max(0, WINDOW - elapsed)
            if sleep_time > 0:
                print(f"Sleeping {sleep_time:.1f}s to avoid rate limit...")
                time.sleep(sleep_time)
            start_time = time.time()
            request_count = 0

# Persist the collected list
with open("matchlist.json", "w") as f:
    json.dump(all_match_ids, f, indent=2)

print(f" Downloaded {len(all_match_ids)} matches to '{RAW_DIR}/'")

PUUIDs fetched: ['T9lGIR8iroZCD7e9-3hWNs-wg9h3eA1UjcT_4YsKlkdZY0L9tZWhJlMg9kGT99wpuBNZxT5iDpY3lg']


 Downloaded 20 matches to 'data/raw/'


### Regenerate matchlist.json after adding new matches

In [6]:
# Regenerate matchlist.json from all match JSON files in data/raw
import os
import json

raw_dir = 'data/raw'
match_ids = []
for fname in os.listdir(raw_dir):
    if fname.endswith('.json') and not fname.endswith('_timeline.json'):
        match_id = fname.replace('.json', '')
        match_ids.append(match_id)

# Remove duplicates, just in case
match_ids = sorted(set(match_ids))

with open("matchlist.json", "w") as f:
    json.dump(match_ids, f, indent=2)

print(f"Updated matchlist.json with {len(match_ids)} match IDs from {raw_dir}")

Updated matchlist.json with 10000 match IDs from data/raw


## Processing & feature extraction

### Design approach (how I actually set this up)



To keep it simple the data is split into two parts based on when the state of the game:



1. **Pregame table** - only info known before the game starts:

   - Champion picks by lane (top, jg, mid, bot, support) for each team.

   - Match duration (just for quick filtering later).



2. **15-minute table** - snapshot of early game:

   - First objective takers (`tower`, `dragon`, `herald`, void grubs = `horde`).

   - Team gold / xp / cs / kills and their diffs at 15:00.

   - Games shorter than 15 minutes are skipped (remakes / noise).



Both tables have one row per `matchId`, so joining them later is trivial.



Why bother splitting? Faster iteration. I can train a draft-only model without loading timeline data, and I can tweak early game logic without touching the pregame export.



Storage:

- Parquet main files: `lol_pregame_data.parquet` and `lol_15min_data.parquet`.

- Small CSV previews (`*_preview.csv`) for a quick look at the dataset



No manifest, no version bumps - if I change logic I just overwrite the files as this is a small scale project. If I later need history I can start versioning.



Flow I run:

1. Regenerate `matchlist.json` from raw if needed.

2. Build pregame (no timeline needed).

3. Build 15m table (needs timeline + duration filter).

4. Save Parquet + preview CSVs.


In [4]:
from scripts.fetch_eune_matches import main as fetch_matches_main

# Runs like: !python scripts/fetch_eune_matches.py --target-match-count 1000 ...
fetch_matches_main([
    "--target-match-count", "1000",
    "--matches-per-player", "100",
    "--history-window", "600",
])

2025-11-03 14:42:46,982 [INFO] Starting with 1749 existing matches
2025-11-03 14:42:50,346 [INFO] No new matches downloaded.
2025-11-03 14:42:50,348 [INFO] Skip summary: {'duplicate_id': 48, 'skip_queue': 16, 'skip_duration': 2}
2025-11-03 14:42:50,346 [INFO] No new matches downloaded.
2025-11-03 14:42:50,348 [INFO] Skip summary: {'duplicate_id': 48, 'skip_queue': 16, 'skip_duration': 2}


0

## Pregame Table

In [ ]:
from scripts.fetch_eune_matches import main as fetch_matches_main

# Runs like: !python scripts/fetch_eune_matches.py --target-match-count 1000 ...
fetch_matches_main([
    "--target-match-count", "10000",
    "--matches-per-player", "100",
    "--history-window", "600",
])

2025-11-03 23:44:49,863 [INFO] Starting with 20 existing matches
2025-11-03 23:44:50,569 [INFO] Stored EUN1_3784733951 (21/10000)
2025-11-03 23:44:50,570 [INFO] Stored EUN1_3784725548 (22/10000)
2025-11-03 23:44:50,570 [INFO] Stored EUN1_3781509782 (23/10000)
2025-11-03 23:44:50,571 [INFO] Stored EUN1_3779740679 (24/10000)
2025-11-03 23:44:50,572 [INFO] Stored EUN1_3779176306 (25/10000)
2025-11-03 23:44:50,572 [INFO] Stored EUN1_3779157935 (26/10000)
2025-11-03 23:44:50,573 [INFO] Stored EUN1_3778613809 (27/10000)
2025-11-03 23:44:50,573 [INFO] Stored EUN1_3778594883 (28/10000)
2025-11-03 23:44:50,574 [INFO] Stored EUN1_3778020386 (29/10000)
2025-11-03 23:44:50,574 [INFO] Stored EUN1_3777886448 (30/10000)
2025-11-03 23:44:50,576 [INFO] Stored EUN1_3777871380 (31/10000)
2025-11-03 23:44:50,576 [INFO] Stored EUN1_3777631308 (32/10000)
2025-11-03 23:44:50,577 [INFO] Stored EUN1_3777615853 (33/10000)
2025-11-03 23:44:50,577 [INFO] Stored EUN1_3777557970 (34/10000)
2025-11-03 23:44:50,578 [

## Fifteen Minute Table

In [1]:
# Setup for 15-minute processing
# Define the target minute and output directory used below
import os

TARGET_MINUTE = 15  # minutes
processed_dir = 'data/processed'
os.makedirs(processed_dir, exist_ok=True)

print(f"TARGET_MINUTE set to {TARGET_MINUTE} minutes")
print(f"Processed output directory: {processed_dir}")


TARGET_MINUTE set to 15 minutes
Processed output directory: data/processed


In [8]:
# Minimal helpers to load raw JSON and aggregate 15m participant stats
import json
from typing import Dict

RAW_DIR = 'data/raw'


def load_match_json(match_id: str) -> Dict:
    path = os.path.join(RAW_DIR, f"{match_id}.json")
    if not os.path.exists(path):
        raise FileNotFoundError(path)
    with open(path, 'r', encoding='utf-8') as f:
        return json.load(f)


def load_timeline_json(match_id: str) -> Dict:
    path = os.path.join(RAW_DIR, f"{match_id}_timeline.json")
    if not os.path.exists(path):
        raise FileNotFoundError(path)
    with open(path, 'r', encoding='utf-8') as f:
        return json.load(f)


def aggregate_participant_stats_at_minute(match_json: Dict, timeline_json: Dict, minute: int = None) -> Dict[int, Dict[str, int]]:
    """Return per-participant stats at the given minute.
    Output shape: {participantId(int): {totalGold, xp, minionsKilled, kills}}
    """
    if minute is None:
        minute = TARGET_MINUTE
    target_ms = minute * 60_000

    frames = timeline_json.get('info', {}).get('frames', [])
    if not frames:
        raise ValueError('Timeline has no frames')

    # Find the latest frame at or before target time
    frame_idx = None
    for i, fr in enumerate(frames):
        if fr.get('timestamp', 0) <= target_ms:
            frame_idx = i
        else:
            break
    if frame_idx is None:
        raise ValueError(f'No frame available at or before {minute} minutes')

    frame = frames[frame_idx]
    part_frames = frame.get('participantFrames') or {}

    # Initialize kills up to target frame from events
    kills = {pid: 0 for pid in range(1, 11)}
    for fr in frames[: frame_idx + 1]:
        for ev in fr.get('events', []) or []:
            if ev.get('type') == 'CHAMPION_KILL':
                kid = ev.get('killerId')
                if isinstance(kid, int) and 1 <= kid <= 10:
                    kills[kid] = kills.get(kid, 0) + 1

    out: Dict[int, Dict[str, int]] = {}
    for k, pf in part_frames.items():
        # Keys may be strings "1".."10"
        try:
            pid = int(k)
        except Exception:
            continue
        tg = pf.get('totalGold')
        xp = pf.get('xp')
        cs = pf.get('minionsKilled')
        # Fallbacks
        tg = 0 if tg is None else tg
        xp = 0 if xp is None else xp
        cs = 0 if cs is None else cs
        out[pid] = {
            'totalGold': int(tg),
            'xp': int(xp),
            'minionsKilled': int(cs),
            'kills': int(kills.get(pid, 0)),
        }

    if not out:
        raise ValueError('participantFrames missing at target minute')
    return out


In [9]:
# Build 15-Minute Table

raw_dir = 'data/raw'
all_match_files = [f.replace('.json', '') for f in os.listdir(raw_dir) if f.endswith('.json') and not f.endswith('_timeline.json')]
print("Number of base match JSON files in data/raw:", len(all_match_files))


# Load full list from matchlist.json
with open('matchlist.json', 'r') as f:
    match_ids_all = json.load(f)
print('Total match IDs from matchlist.json:', len(match_ids_all))


MIN_GAME_DURATION = TARGET_MINUTE * 60  # require full duration to compute 15m stats
rows = []
skipped_short = []
missing_timeline = []
skipped_snapshot = []

# Process each match
for match_id in tqdm(match_ids_all, desc="Processing matches (15m)"):
    try:
        match_json = load_match_json(match_id)
    except FileNotFoundError:
        continue  # skip if match file missing
    game_duration = match_json.get('info', {}).get('gameDuration', 0)
    if game_duration < MIN_GAME_DURATION:  # ignore super short games (remakes)
        skipped_short.append((match_id, game_duration))
        continue
    try:
        timeline_json = load_timeline_json(match_id)  # need events for early stats
    except FileNotFoundError:
        missing_timeline.append(match_id)
        continue
    try:
        stats_snapshot = aggregate_participant_stats_at_minute(match_json, timeline_json)
    except ValueError as exc:
        skipped_snapshot.append((match_id, str(exc)))
        continue
    teams = match_json['info']['teams']



# Method to determine which team got first objective
def first_team(obj_key):
    try:
        if teams[0]['objectives'][obj_key]['first']:
            return 'blue'
        if teams[1]['objectives'][obj_key]['first']:
            return 'red'
    except Exception:
        return None
    return None

# base row
row = {
    'matchId': match_id,
    'queueId': match_json['info'].get('queueId'),
    'gameDuration': game_duration,
    'blue_win': int(match_json['info']['teams'][0]['win']),  # 1 if blue won
    'first_tower': first_team('tower'),
    'first_dragon': first_team('dragon'),
    'first_herald': first_team('riftHerald'),
    'first_grub': first_team('horde'),  # void grubs are exposed as HORDE in the API
}

# participantId lists split by side
blue_ids = [p['participantId'] for p in match_json['info']['participants'][:5]]
red_ids = [p['participantId'] for p in match_json['info']['participants'][5:]]

# aggregate team sums at 15m
row.update({
    'blue_gold_15': sum(stats_snapshot[pid]['totalGold'] for pid in blue_ids),
    'red_gold_15': sum(stats_snapshot[pid]['totalGold'] for pid in red_ids),
    'blue_xp_15': sum(stats_snapshot[pid]['xp'] for pid in blue_ids),
    'red_xp_15': sum(stats_snapshot[pid]['xp'] for pid in red_ids),
    'blue_cs_15': sum(stats_snapshot[pid]['minionsKilled'] for pid in blue_ids),
    'red_cs_15': sum(stats_snapshot[pid]['minionsKilled'] for pid in red_ids),
    'blue_kills_15': sum(stats_snapshot[pid]['kills'] for pid in blue_ids),
    'red_kills_15': sum(stats_snapshot[pid]['kills'] for pid in red_ids),
})

# diffs (blue - red)
row['gold_diff_15'] = row['blue_gold_15'] - row['red_gold_15']
row['cs_diff_15'] = row['blue_cs_15'] - row['red_cs_15']
row['xp_diff_15'] = row['blue_xp_15'] - row['red_xp_15']
row['kills_diff_15'] = row['blue_kills_15'] - row['red_kills_15']
rows.append(row)

# Build DataFrame
df_15m = pd.DataFrame(rows)

# Save outputs (full + small preview)
fifteen_parquet = os.path.join(processed_dir, 'lol_15min_data.parquet')
fifteen_csv = os.path.join(processed_dir, 'lol_15min_data_preview.csv')
df_15m.to_parquet(fifteen_parquet, index=False)
df_15m.head(50).to_csv(fifteen_csv, index=False)
print('Saved 15-minute dataset files (Parquet + preview CSV).')

Number of base match JSON files in data/raw: 10000
Total match IDs from matchlist.json: 10000


Processing matches (15m):   9%|▊         | 860/10000 [00:40<07:11, 21.18it/s]



KeyboardInterrupt: 

## Fix: Fifteen Minute Table (corrected loop)
The previous cell likely had an indentation/scope issue: row construction happened outside the `for match_id in ...` loop, so only the last match produced a row. The cell below builds each `row` and appends it to `rows` inside the loop, and also prints a summary of kept and skipped matches.

In [10]:
# Rebuild 15-minute table with correct loop placement
import os
import pandas as pd
from tqdm import tqdm

# You can set SAMPLE_N=None to process all matches; keep a small sample first for a quick sanity-check
SAMPLE_N = None

MIN_GAME_DURATION = TARGET_MINUTE * 60
rows = []
skipped_short, missing_timeline, skipped_snapshot = [], [], []

ids_iter = match_ids_all
if SAMPLE_N is not None and SAMPLE_N > 0:
    ids_iter = match_ids_all[:SAMPLE_N]

for match_id in tqdm(ids_iter, desc=f"Processing matches (15m, SAMPLE_N={SAMPLE_N})"):
    try:
        match_json = load_match_json(match_id)
    except FileNotFoundError:
        continue

    game_duration = match_json.get('info', {}).get('gameDuration', 0)
    if game_duration < MIN_GAME_DURATION:
        skipped_short.append((match_id, game_duration))
        continue

    try:
        timeline_json = load_timeline_json(match_id)
    except FileNotFoundError:
        missing_timeline.append(match_id)
        continue

    try:
        stats_snapshot = aggregate_participant_stats_at_minute(match_json, timeline_json, minute=TARGET_MINUTE)
    except ValueError as exc:
        skipped_snapshot.append((match_id, str(exc)))
        continue

    teams = match_json['info']['teams']

    def first_team(obj_key):
        try:
            if teams[0]['objectives'][obj_key]['first']:
                return 'blue'
            if teams[1]['objectives'][obj_key]['first']:
                return 'red'
        except Exception:
            return None
        return None

    row = {
        'matchId': match_id,
        'queueId': match_json['info'].get('queueId'),
        'gameDuration': game_duration,
        'blue_win': int(teams[0]['win']),
        'first_tower': first_team('tower'),
        'first_dragon': first_team('dragon'),
        'first_herald': first_team('riftHerald'),
        'first_grub': first_team('horde'),
    }

    blue_ids = [p['participantId'] for p in match_json['info']['participants'][:5]]
    red_ids = [p['participantId'] for p in match_json['info']['participants'][5:]]

    row.update({
        'blue_gold_15': sum(stats_snapshot.get(pid, {}).get('totalGold', 0) for pid in blue_ids),
        'red_gold_15': sum(stats_snapshot.get(pid, {}).get('totalGold', 0) for pid in red_ids),
        'blue_xp_15': sum(stats_snapshot.get(pid, {}).get('xp', 0) for pid in blue_ids),
        'red_xp_15': sum(stats_snapshot.get(pid, {}).get('xp', 0) for pid in red_ids),
        'blue_cs_15': sum(stats_snapshot.get(pid, {}).get('minionsKilled', 0) for pid in blue_ids),
        'red_cs_15': sum(stats_snapshot.get(pid, {}).get('minionsKilled', 0) for pid in red_ids),
        'blue_kills_15': sum(stats_snapshot.get(pid, {}).get('kills', 0) for pid in blue_ids),
        'red_kills_15': sum(stats_snapshot.get(pid, {}).get('kills', 0) for pid in red_ids),
    })

    row['gold_diff_15'] = row['blue_gold_15'] - row['red_gold_15']
    row['cs_diff_15'] = row['blue_cs_15'] - row['red_cs_15']
    row['xp_diff_15'] = row['blue_xp_15'] - row['red_xp_15']
    row['kills_diff_15'] = row['blue_kills_15'] - row['red_kills_15']

    rows.append(row)

# Build DataFrame and save
fixed_df_15m = pd.DataFrame(rows)
print(
    f"Processed: {len(ids_iter)} | Kept: {len(fixed_df_15m)} | "
    f"Skipped<15m: {len(skipped_short)} | MissingTimeline: {len(missing_timeline)} | SnapshotErrors: {len(skipped_snapshot)}"
)

# Only write if we have rows
if len(fixed_df_15m) > 0:
    fifteen_parquet = os.path.join(processed_dir, 'lol_15min_data.parquet')
    fifteen_csv = os.path.join(processed_dir, 'lol_15min_data_preview.csv')
    fixed_df_15m.to_parquet(fifteen_parquet, index=False)
    fixed_df_15m.head(50).to_csv(fifteen_csv, index=False)
    print('Saved 15-minute dataset files (Parquet + preview CSV) [corrected loop].')
else:
    print('No rows to write; check skip counters above.')


Processing matches (15m, SAMPLE_N=None): 100%|██████████| 10000/10000 [07:20<00:00, 22.71it/s]



Processed: 10000 | Kept: 9997 | Skipped<15m: 3 | MissingTimeline: 0 | SnapshotErrors: 0
Saved 15-minute dataset files (Parquet + preview CSV) [corrected loop].
